In [1]:
%load_ext autoreload
%autoreload 2

In [70]:
import glob
import torch
import os,sys
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sb
import pandas as pd
import dill
import flow_ssl
from sklearn.model_selection import train_test_split
from flow_ssl.data import GAS, HEPMASS, MINIBOONE
from flow_ssl.data.nlp_datasets import AG_News,YAHOO
from torch.utils.data import DataLoader
#from oil.datasetup.dataloaders import getLabLoader,classBalancedSampleIndices
from oil.datasetup.datasets import split_dataset
from oil.utils.utils import FixedNumpySeed
from oil.model_trainers.graphssl import GraphSSL

In [5]:
H = HEPMASS(os.path.expanduser('~/datasets/UCI/hepmass/'),train=True)
Ht = HEPMASS(os.path.expanduser('~/datasets/UCI/hepmass/'),train=False)
M = MINIBOONE(os.path.expanduser('~/datasets/UCI/miniboone/'),train=True)
Mt = MINIBOONE(os.path.expanduser('~/datasets/UCI/miniboone/'),train=False)
A = AG_News(os.path.expanduser('~/datasets/AGNEWS/'),train=True)
At = AG_News(os.path.expanduser('~/datasets/AGNEWS/'),train=False)
Ya = YAHOO(os.path.expanduser('~/datasets/YAHOO/'),train=True)
Yt = YAHOO(os.path.expanduser('~/datasets/YAHOO/'),train=False)

748342
140000


In [6]:
def oh(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [5]:
# train_data = np.load('ag_news_train.npz')
# x_train = train_data['encodings']
# y_train = train_data['labels']
# test_data = np.load('ag_news_test.npz')
# x_test = test_data['encodings']
# y_test = test_data['labels']

In [7]:
test_size = 5000#3000

unlab_size=20000#5000

In [121]:
X_train = H.trn
X_test = H.tst
y_train = H.y_trn.astype(int)
y_test = H.y_tst.astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#
lab_size = 20
Ds = H

In [127]:
X_train = M.X_train
X_test = M.X_test
y_train = M.y_train.astype(int)
y_test = M.y_test.astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#
lab_size = 20
Ds = M

In [132]:

X_train = A.train_data.numpy()
m,s = X_train.mean(0),X_train.std(0)
X_test = At.test_data.numpy()#[:test_size]
y_train = A.train_labels.numpy().astype(int)
y_test = At.test_labels.numpy().astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_train = (X_train-m)/s
X_test = (X_test-m)/s
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#

lab_size = 200
Ds = A

In [138]:
X_train = Ya.X_train
m,s = X_train.mean(0),X_train.std(0)
X_test = Ya.X_test
y_train = Ya.y_train.astype(int)
y_test = Ya.y_test.astype(int)
test_indices = np.random.choice(y_test.shape[0],test_size,replace=False)
X_train = (X_train-m)/s
X_test = (X_test-m)/s
X_test_small = X_test[test_indices]
y_test_small = y_test[test_indices]#

lab_size = 800
Ds = Ya

In [14]:
trainset

'val'

In [139]:
with FixedNumpySeed(0):
    datasets = split_dataset(Ds,splits={'train':lab_size,'val':5000})

X_lab = np.stack([datasets['train'][i][0] for i in range(len(datasets['train']))],axis=0)
y_lab = np.stack([datasets['train'][i][1] for i in range(len(datasets['train']))],axis=0)
X_dev = np.stack([datasets['val'][i][0] for i in range(len(datasets['val']))],axis=0)
y_dev = np.stack([datasets['val'][i][1] for i in range(len(datasets['val']))],axis=0)
X_unlab = np.stack([Ds[i][0] for i in range(len(Ds))],axis=0)
y_unlab = np.stack([Ds[i][1] for i in range(len(Ds))],axis=0)
#     labIndices, devIndices = classBalancedSampleIndices(Ds, lab_size+5000, 5000)
#     unlab_indices = np.random.choice(y_train.shape[0],unlab_size,replace=False)
# X_lab, X_dev = X_train[labIndices], X_train[devIndices]
# y_lab, y_dev = y_train[labIndices], y_train[devIndices]
# X_unlab = X_train[unlab_indices]
# y_unlab = y_train[unlab_indices]

In [131]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_lab, y_lab)
clf.score(X_test, y_test)

0.7897260273972603

In [134]:
unlab_indices = np.random.choice(X_unlab.shape[0],1000,replace=False)
y_unlaba = y_unlab[unlab_indices]
X_unlaba = X_unlab[unlab_indices]
Y = np.concatenate((y_lab,y_unlaba))
Y[len(y_lab):] =-1
dev_accs = []
hypers = []
for gamma in [2,4,8,10]:
    for reg in [1/16,1/8,1/4,1/2]:
        graphssl = GraphSSL(reg=reg,gamma=gamma)
        graphssl.fit(np.concatenate((X_lab,X_unlaba)), Y)
        dev_accs.append(graphssl.score(X_dev,y_dev))
        hypers.append((gamma,reg))
best_gamma,best_reg = hypers[np.argmax(dev_accs)]
graphssl = GraphSSL(reg=best_reg,gamma=best_gamma)
graphssl.fit(np.concatenate((X_lab,X_unlaba)), Y)
test_acc = graphssl.score(X_test,y_test)
print(f"Best of {test_acc} with gamma={best_gamma}, reg={best_reg}")

Best of 0.35486842105263156 with gamma=10, reg=0.5


In [ ]:
hypers[np.argmax(dev_accs)]

In [130]:
unlab_indices = np.random.choice(X_unlab.shape[0],5000,replace=False)
y_unlaba = y_unlab[unlab_indices]
X_unlaba = X_unlab[unlab_indices]
Y = np.concatenate((y_lab,y_unlaba))
Y[len(y_lab):] =-1
graphssl = GraphSSL(reg=.5,gamma=4)
#print("constructed")
graphssl.fit(np.concatenate((X_lab,X_unlaba)), Y)
print(graphssl.score(X_test,y_test))

0.7936986301369863


In [ ]:
(1-graphssl.predict(X_test_small)==y_test_small).mean()

In [ ]:
X = np.concatenate((X_lab,X_unlab,X_test_small))
# l = np.arange(200)
# u = np.arange(200,1200)
l = np.arange(X.shape[0])<lab_size
u = (~l)&(np.arange(X.shape[0])<lab_size+X_unlab.shape[0])
t = np.arange(X.shape[0])>=lab_size+X_unlab.shape[0]


In [ ]:
s=1/2
gamma=4
#dists = np.sum((X[:,None,:]-X[None,:,:])**2,axis=-1)/s**2
nx = np.linalg.norm(X,axis=1)
dists = 1-(X@X.T)/(nx[:,None]*nx[None,:])
W = np.exp(-dists*gamma)#-dists*100
W -= np.diag(np.diag(W))
#W += .00001*np.eye(X.shape[0])
D = np.diag(np.sum(W,axis=-1))
dm2 = np.sum(W,axis=-1)**-.5


In [ ]:
L =dm2[:,None]*(D-s*W)*dm2[None,:]
Y = np.zeros((X.shape[0],Ds.num_classes))
Y[:len(y_lab)] = oh(y_lab,Ds.num_classes)
Ys = np.linalg.solve(L,Y)
(Ys[t].argmax(-1)==y_test_small).mean()

In [ ]:
fu = -np.linalg.solve(L[u|t][:,u|t],(L[u|t][:,l]@oh(y_lab,Ds.num_classes)))
(fu[-X_test_small.shape[0]:].argmax(-1)==y_test_small).mean()

# Fast kNN Ball Tree & Sparse Label Propagation

In [135]:
import sklearn
def normalize(X):
    return X/np.linalg.norm(X,axis=1)[:,None]

In [136]:
# k=5
# tree = sklearn.neighbors.BallTree(lab_and_unlab)
# dists, indices = tree.query(lab_and_unlab,k=k)

In [140]:
scores = []
kvals = list(range(1,20))
for k in kvals:
    knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k)
    knn.fit(normalize(X_lab),y_lab)
    scores.append(knn.score(normalize(X_dev), y_dev))

best_k = kvals[np.argmax(scores)]
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=best_k)
knn.fit(normalize(X_lab),y_lab)
test_acc = knn.score(normalize(X_test), y_test)
print(f"Best test acc of {test_acc} with k={best_k}")

Best test acc of 0.1978 with k=19


In [59]:
import sklearn.semi_supervised
k=3
# partially_labels = np.concatenate((y_lab,y_unlab))
# partially_labels[len(y_lab):] =-1
# LP = sklearn.semi_supervised.LabelPropagation(kernel='knn',gamma=gamma,n_neighbors=k)
# LP.fit(normalize(lab_and_unlab),partially_labels)
# LP.score(normalize(X_dev),y_dev)

In [88]:
unlab_indices = np.random.choice(X_unlab.shape[0],10000,replace=False)
y_unlaba = y_unlab[unlab_indices]
X_unlaba = X_unlab[unlab_indices]
Y = np.concatenate((y_lab,y_unlaba))
lab_and_unlab = np.concatenate((X_lab,X_unlaba))
partially_labels = np.concatenate((y_lab,y_unlaba))
partially_labels[len(y_lab):] =-1
dev_accs = []
hypers = []
for alpha in [.1]:#,.3,.5]:
    for k in [16]:#,24]:
        label_prop_model = sklearn.semi_supervised.LabelSpreading(kernel = 'knn',
                                                                  alpha=alpha,n_neighbors=k)
        label_prop_model.fit(normalize(lab_and_unlab), partially_labels)
        dev_accs.append(label_prop_model.score(normalize(X_dev),y_dev))
        hypers.append((alpha,k))
best_alpha,best_k = hypers[np.argmax(dev_accs)]
label_prop_model = sklearn.semi_supervised.LabelSpreading(kernel = 'knn',
                                        alpha=best_alpha,n_neighbors=best_k)
label_prop_model.fit(normalize(lab_and_unlab), partially_labels)
test_acc = label_prop_model.score(normalize(X_test),y_test)
print(f"Best of {test_acc} with alpha={best_alpha}, k={best_k}")

/home/marc_f/anaconda3/envs/default/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
/home/marc_f/anaconda3/envs/default/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


Best of 0.6552054794520548 with alpha=0.1, k=16


In [89]:
lab_and_unlab = np.concatenate((X_lab,X_unlaba))
partially_labels = np.concatenate((y_lab,y_unlaba))
partially_labels[len(y_lab):] =-1
best_alpha,best_k = hypers[np.argmax(dev_accs)]
label_prop_model = sklearn.semi_supervised.LabelSpreading(kernel = 'knn',
                                        alpha=.1,n_neighbors=24)
label_prop_model.fit(normalize(lab_and_unlab), partially_labels)
test_acc = label_prop_model.score(normalize(X_test),y_test)
print(f"Best of {test_acc} with alpha={best_alpha}, k={best_k}")

/home/marc_f/anaconda3/envs/default/lib/python3.6/site-packages/sklearn/semi_supervised/label_propagation.py:293: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


Best of 0.6412328767123288 with alpha=0.1, k=16


In [ ]:
plt.bar(range(10),[(partially_labels==i).sum() for i in range(10)])

In [ ]:
1

In [ ]:
X = np.concatenate((X_lab,X_unlab))
nx = np.linalg.norm(X,axis=1)
dists = 1-(X@X.T)/(nx[:,None]*nx[None,:])
W = np.exp(-dists*2)#-dists*100
W -= np.diag(np.diag(W))
#W += .00001*np.eye(X.shape[0])
D = np.diag(np.sum(W,axis=-1))
dm2 = np.sum(W,axis=-1)**-.5
L =dm2[:,None]*(D-W)*dm2[None,:]


In [ ]:

nxtest = np.linalg.norm(X_test_small,axis=1)
dists = 1-(X@X_test_small.T)/(nx[:,None]*nxtest[None,:])
W2 = np.exp(-dists*2)#-dists*100
#W += .00001*np.eye(X.shape[0])
dm2_ = np.sum(W2,axis=0)**-.5
L2 =dm2[:,None]*W2*dm2_[None,:]

In [ ]:
Y = np.zeros((X.shape[0],Ds.num_classes))
Y[:len(y_lab)] = oh(y_lab,Ds.num_classes)
Ys = np.linalg.solve(L+np.eye(X.shape[0]),Y)

In [ ]:
((L2.T@graphssl.Ys).argmax(-1)==y_test_small).mean()

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_lab, y_lab)
clf.score(X_test, y_test)

In [ ]:
def ik(X1,X2):
    X1n = X1/np.linalg.norm(X1,axis=1)[:,None]
    X2n = X2/np.linalg.norm(X2,axis=1)[:,None]
    return np.exp(-2*(1-X1n@X2n.T))

In [ ]:


from sklearn.semi_supervised import LabelSpreading

partially_labels = np.concatenate((y_lab,y_unlab))
partially_labels[len(y_lab):] =-1
label_prop_model = LabelSpreading(kernel = ik,alpha=.0001)
label_prop_model.fit(np.concatenate((X_lab,X_unlab)), partially_labels)
label_prop_model.score(X_test,y_test)

In [ ]:
from oil.model_trainers.graphssl import GraphSSL
graphssl = GraphSSL()
graphssl.fit(np.concatenate((X_lab,X_unlab)), partially_labels)
graphssl.score(X_test,y_test)

In [ ]:
X_train.shape

In [ ]:
fu[-X_test.shape[0]:].argmax(-1)==y_test

In [ ]:
y_test.shape

In [ ]:
fu[-X_test.shape[0]:].argmax(-1).shape

In [ ]:
fu.shape

In [ ]:
np.where(u|t)

In [ ]:
np.where(t)

In [ ]:
t.shape

In [ ]:
fu[t].argmax(-1)==y_l

In [ ]:
+0*np.eye(X.shape[0]-lab_size)

In [ ]:
beta = np.linalg.solve(X[l].T@X[l]+100*np.eye(X.shape[1]),X[l].T@oh(y_lab,4))

In [ ]:
((X[l]@beta).argmax(-1)==y_lab).mean()

In [ ]:
((X[u]@beta).argmax(-1)==y_unlab).mean()

In [ ]:
((((x_test-m)/s)@beta).argmax(-1)==y_test).mean()